In [64]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import psutil, os

### Initial Analysis of Calendar File

In [2]:
# Read in Calendar file
df_calendar = pd.read_csv('/Users/ianforrest/Downloads/m5-forecasting-accuracy/calendar.csv')

# convert NaN values to 'None'; NaN values only contained in event_name_# & event_type_#, where NaN represents 'None'
df_calendar = df_calendar.fillna('None')

# create list of all unique event_names in dataset
event_name_list = set(df_calendar['event_name_1'].unique().tolist() + df_calendar['event_name_2'].unique().tolist())
event_name_list = list(event_name_list)
# convert to dictionary
event_name_dict = {ni: indi for indi, ni in enumerate(set(event_name_list))}
# adjust value of 'None' to be uniform across all 'name' & 'type' columns
event_name_dict['None'] = 0
event_name_dict['LentStart'] = 18
# map to dataframe
df_calendar['event_name_1'] = df_calendar['event_name_1'].map(event_name_dict)
df_calendar['event_name_2'] = df_calendar['event_name_2'].map(event_name_dict)

# create list of all unique event_types in dataset
event_type_list = set(df_calendar['event_type_1'].unique().tolist() + df_calendar['event_type_2'].unique().tolist())
event_type_list = list(event_type_list)
# convert to dictionary
event_type_dict = {ni: indi for indi, ni in enumerate(set(event_type_list))}
# adjust value of 'None' to be uniform across all 'name' & 'type' columns
event_type_dict['None'] = 0
event_type_dict['National'] = 1
# map to dataframe
df_calendar['event_type_1'] = df_calendar['event_type_1'].map(event_type_dict)
df_calendar['event_type_2'] = df_calendar['event_type_2'].map(event_type_dict)

# adjust 'd' column, change name to 'day_total'
df_calendar = df_calendar.rename(columns={"d": "day_total", "wday":"day_of_wk"})
df_calendar['day_total'] = df_calendar['day_total']

# convert category columns to 'category' dtype
category_columns = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'weekday', 'day_of_wk']
for column in category_columns:
    df_calendar[column] = df_calendar[column].astype('category')

# convert 'date' column to datetime
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
# add 'day_of_month' column
df_calendar['day_of_month'] = df_calendar['date'].map(lambda x: x.day)
# create custom data dictionary for day_of_wk column columns
day_dict = {'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4,
            'Thursday':5, 'Friday':6, 'Saturday':7}
# map to dataframe
df_calendar['day_of_wk'] = df_calendar['weekday'].map(day_dict)
# drop 'weekday' column, now contains redundant information
df_calendar = df_calendar.drop(columns = ['weekday'])

# rearrange columns
df_calendar = df_calendar[['date', 'wm_yr_wk', 'day_of_wk', 'day_of_month',
                           'month', 'year', 'day_total', 'event_name_1', 'event_type_1', 'event_name_2',
                           'event_type_2','snap_CA', 'snap_TX', 'snap_WI']]

df_calendar.head()

,date,wm_yr_wk,day_of_wk,day_of_month,month,year,day_total,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,7,29,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,1,30,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,2,31,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,3,1,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,4,2,2,2011,d_5,0,0,0,0,1,0,1


In [3]:
df_calendar.dtypes

date            datetime64[ns]
wm_yr_wk                 int64
day_of_wk                int64
day_of_month             int64
month                    int64
year                     int64
day_total               object
event_name_1          category
event_type_1          category
event_name_2          category
event_type_2          category
snap_CA                  int64
snap_TX                  int64
snap_WI                  int64
dtype: object

In [4]:
df_calendar.shape

(1969, 14)

In [8]:
df_calendar.tail()

,date,wm_yr_wk,day_of_wk,day_of_month,month,year,day_total,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,4,15,6,2016,d_1965,0,0,0,0,0,1,1
1965,2016-06-16,11620,5,16,6,2016,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,6,17,6,2016,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,7,18,6,2016,d_1968,0,0,0,0,0,0,0
1968,2016-06-19,11621,1,19,6,2016,d_1969,22,4,7,2,0,0,0


### Initial Analysis of Sales_Train_Val File
- may have to drop last 56 rows from df_calendar
- df_calendar accounts for 1969 days, while df_sales_train_val only accounts for 1913 days
- need to be uniform for merge

In [32]:
# define categories
numcols = [f"d_{day}" for day in range(1,1914)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numcol:"float32" for numcol in numcols} 
dtype.update({col: "category" for col in catcols if col != "id"})

# read in csv
df_sales_train_val = pd.read_csv('/Users/ianforrest/Downloads/m5-forecasting-accuracy/sales_train_validation.csv', 
                                usecols = catcols + numcols, dtype = dtype)
df_sales_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0


In [33]:
# extend d_#### to match calendar file
for day in range(1914, 1970):
    df_sales_train_val[f"d_{day}"] = np.nan
    
df_sales_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# encode item_id, dept_id, cat_id, store_id, & state_id to categorical variables
# doesn't need to be as complex as df_calendar items
for col in catcols:
    if col != "id":
        df_sales_train_val[col] = df_sales_train_val[col].cat.codes.astype("int16")
        df_sales_train_val[col] -= df_sales_train_val[col].min()

In [35]:
df_sales_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# melt dataframe down
df_sales_train_val = pd.melt(df_sales_train_val, id_vars = catcols, value_vars = [col for col in df_sales_train_val.columns if col.startswith("d_")],
             var_name = "day_total", value_name = "sales")

In [37]:
df_sales_train_val.head()

,id,item_id,dept_id,store_id,cat_id,state_id,day_total,sales
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1,0.0
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1,0.0
2,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1,0.0
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1,0.0
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_1,0.0


In [38]:
df_sales_train_val.dtypes

id            object
item_id        int16
dept_id        int16
store_id       int16
cat_id         int16
state_id       int16
day_total     object
sales        float64
dtype: object

In [39]:
# attempt merge
df_sales_train_val = df_sales_train_val.merge(df_calendar, on= "day_total", copy = False)

In [40]:
df_sales_train_val.head()

,id,item_id,dept_id,store_id,cat_id,state_id,day_total,sales,date,wm_yr_wk,...,day_of_month,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1,0.0,2011-01-29,11101,...,29,1,2011,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_1,0.0,2011-01-29,11101,...,29,1,2011,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1,0.0,2011-01-29,11101,...,29,1,2011,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1,0.0,2011-01-29,11101,...,29,1,2011,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_1,0.0,2011-01-29,11101,...,29,1,2011,0,0,0,0,0,0,0


In [41]:
df_sales_train_val.shape

(60034810, 21)

#### Initial Analysis of Sell Prices

In [43]:
sell_price_dtypes = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
df_sell_prices = pd.read_csv('/Users/ianforrest/Downloads/m5-forecasting-accuracy/sell_prices.csv',
                             dtype = sell_price_dtypes)
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [44]:
df_sell_prices.dtypes

store_id      category
item_id       category
wm_yr_wk         int16
sell_price     float32
dtype: object

In [45]:
# encode categorical variables
for col, col_dtype in sell_price_dtypes.items():
    if col_dtype == "category":
        df_sell_prices[col] = df_sell_prices[col].cat.codes.astype("int16")
        df_sell_prices[col] -= df_sell_prices[col].min()

In [46]:
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.58
1,0,0,11326,9.58
2,0,0,11327,8.26
3,0,0,11328,8.26
4,0,0,11329,8.26


In [47]:
# merge with train_val
df_sales_train_val = df_sales_train_val.merge(df_sell_prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [50]:
df_sales_train_val.tail()

,id,item_id,dept_id,store_id,cat_id,state_id,day_total,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
47735392,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1969,NaN,2016-06-19,11621,...,6,2016,22,4,7,2,0,0,0,3.98
47735393,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1968,NaN,2016-06-18,11621,...,6,2016,0,0,0,0,0,0,0,1.28
47735394,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1969,NaN,2016-06-19,11621,...,6,2016,22,4,7,2,0,0,0,1.28
47735395,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1968,NaN,2016-06-18,11621,...,6,2016,0,0,0,0,0,0,0,1.00
47735396,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1969,NaN,2016-06-19,11621,...,6,2016,22,4,7,2,0,0,0,1.00


In [49]:
df_sales_train_val.shape

(47735397, 22)

# Memory Reduce exploration

In [65]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

get_memory_usage()

5.68

In [56]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [57]:
df = reduce_mem_usage(df_sales_train_val, verbose=True)

Mem. usage decreased to 2549.35 Mb (54.1% reduction)


In [58]:
# impute missing values in 'sales' column
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)

In [60]:
# impute sales based on date
imp.fit_transform([df['sales'], df['month']])

MemoryError: 

# Sample Sub

In [11]:
df_sample_sub = pd.read_csv('/Users/ianforrest/Downloads/m5-forecasting-accuracy/sample_submission.csv')
df_sample_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_sample_sub.shape

(60980, 29)